In [1]:
!pip install llama-stack -U
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir
!apt-get install -y cmake build-essential

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=7efa078b50f9e0abfcb57dc164733efd067262c9d4b0904ccb6df7739c7afda6
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 287.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 146.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
!llama model list --show-all

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Model Descriptor(ID)         ┃ Hugging Face Repo            ┃ Context Length ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Llama-2-7b                   │ meta-llama/Llama-2-7b        │ 4K             │
├──────────────────────────────┼──────────────────────────────┼────────────────┤
│ Llama-2-13b                  │ meta-llama/Llama-2-13b       │ 4K             │
├──────────────────────────────┼──────────────────────────────┼────────────────┤
│ Llama-2-70b                  │ meta-llama/Llama-2-70b       │ 4K             │
├──────────────────────────────┼──────────────────────────────┼────────────────┤
│ Llama-2-7b-chat              │ meta-llama/Llama-2-7b-chat   │ 4K             │
├──────────────────────────────┼──────────────────────────────┼────────────────┤
│ Llama-2-13b-chat             │ meta-llama/Llama-2-13b-chat  │ 4K             │
├───────────────────────────

Convertir de .pth a gguf

In [3]:
# Paso 1: Instalar herramientas necesarias
!pip install -q llama-stack transformers sentencepiece

# Paso 2: Descargar el modelo desde Meta (solo si no lo tienes ya)
!llama model download --source meta --model-id Llama3.2-3B-Instruct

# Paso 3: Preparar los archivos del modelo para trabajar con ellos
!mkdir -p /content/models/Llama3.2-3B-Instruct
!cp /root/.llama/checkpoints/Llama3.2-3B-Instruct/* /content/models/Llama3.2-3B-Instruct/

Please provide the signed URL for model Llama3.2-3B-Instruct you received via email after visiting https://www.llama.com/llama-downloads/ (e.g., https://llama3-1.llamameta.net/*?Policy...): object address  : 0x7e4205d9efe0
object refcount : 2
object type     : 0x9d7580
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C
cp: cannot stat '/root/.llama/checkpoints/Llama3.2-3B-Instruct/*': No such file or directory


In [ ]:
# Paso 4: Clonar llama.cpp y compilarlo
%cd /content
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!cmake -S . -B build
!cmake --build build

In [ ]:
# Paso 5: Convertir el modelo .pth al formato Hugging Face
!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/llama/convert_llama_weights_to_hf.py

!python3 convert_llama_weights_to_hf.py \
  --input_dir /content/models/Llama3.2-3B-Instruct \
  --model_size 3B \
  --output_dir /content/models/Llama3.2-3B-Instruct-HF \
  --llama_version 3


In [ ]:
!ls -lh /content/models/Llama3.2-3B-Instruct-HF/


In [ ]:
!wget https://raw.githubusercontent.com/ggerganov/llama.cpp/master/convert_hf_to_gguf.py -O convert_hf_to_gguf.py



In [ ]:
!python3 convert_hf_to_gguf.py \
  /content/models/Llama3.2-3B-Instruct-HF \
  --outfile /content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf \
  --outtype f16


In [ ]:
!pip install llama-cpp-python==0.2.61 gradio --quiet
!pip uninstall -y llama-cpp-python
!pip install llama-cpp-python --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/avx2


In [ ]:
from llama_cpp import Llama
import gradio as gr

# Ruta al modelo
MODEL_PATH = "/content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf"

# Cargar el modelo
#llm = Llama(
 #   model_path=MODEL_PATH,
 #   n_ctx=4096,
  #  n_threads=8,  # Puedes bajar a 4 si va lento
   # n_gpu_layers=35  # Usa la GPU si tienes activado aceleración
#)
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,          # Menos contexto = menos memoria
    n_threads=4,         # Reduce hilos para no saturar CPU
    n_gpu_layers=26      # Baja a 20 si estás en GPU pequeña (o 0 si estás sin GPU)
)

# Función de respuesta
def responder(mensaje, historia):
    prompt = f"{mensaje}\nAssistant:"
    respuesta = llm(prompt, max_tokens=256, stop=["\n", "</s>"])
    texto = respuesta["choices"][0]["text"].strip()
    historia.append((mensaje, texto))
    return historia, historia


In [ ]:
# Interfaz tipo chat
chatbot = gr.ChatInterface(
    fn=responder,
    title="Asistente LLaMA 3.2 3B Instruct",
    chatbot=gr.Chatbot(),
    textbox=gr.Textbox(placeholder="Haz una pregunta...", lines=2),
    examples=["¿Qué es la computación cuántica?", "Explícame la Segunda Guerra Mundial"],
    cache_examples=False
)

# Ejecutar la app
chatbot.launch()

___________________________________

In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf",
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=26  # Usa GPU para las primeras 26 capas
)

output = llm("Dime una curiosidad sobre Marte.", max_tokens=64)
print(output["choices"][0]["text"])


--------------------------------------------------------------------------------

In [ ]:
# -----------------------------------------
# INSTALAR DEPENDENCIAS Y PREPARAR ENTORNO
# -----------------------------------------
!apt-get -y install cmake build-essential
!pip install -q llama-stack transformers sentencepiece
!pip install -q llama-cpp-python==0.2.61 gradio
!pip uninstall -y llama-cpp-python
!pip install llama-cpp-python --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/avx2

# -----------------------------------------
# DESCARGAR Y PREPARAR MODELO
# -----------------------------------------
!llama model download --source meta --model-id Llama3.2-3B-Instruct

!mkdir -p /content/models/Llama3.2-3B-Instruct
!cp /root/.llama/checkpoints/Llama3.2-3B-Instruct/* /content/models/Llama3.2-3B-Instruct/

# -----------------------------------------
# CONVERTIR A FORMATO HUGGING FACE
# -----------------------------------------
!cd /content && git clone https://github.com/ggerganov/llama.cpp.git
!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/llama/convert_llama_weights_to_hf.py

!python3 convert_llama_weights_to_hf.py \
  --input_dir /content/models/Llama3.2-3B-Instruct \
  --model_size 3B \
  --output_dir /content/models/Llama3.2-3B-Instruct-HF \
  --llama_version 3

# -----------------------------------------
# CONVERTIR A GGUF
# -----------------------------------------
!wget https://raw.githubusercontent.com/ggerganov/llama.cpp/master/convert_hf_to_gguf.py -O convert_hf_to_gguf.py

!python3 convert_hf_to_gguf.py \
  /content/models/Llama3.2-3B-Instruct-HF \
  --outfile /content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf \
  --outtype f16


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf",
    n_ctx=1024,
    n_threads=4,
    n_gpu_layers=0  # 0 = todo en CPU (para confirmar si el fallo es GPU)
)

respuesta = llm("¿Cuál es la capital de Japón?", max_tokens=64)
print(respuesta["choices"][0]["text"])


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/models/Llama3.2-3B-Instruct/llama-3.2b-instruct.gguf",
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=28  # Full GPU para modelo 3B en Colab T4
)

output = llm("Dime una curiosidad sobre Marte.", max_tokens=64)
print("Respuesta:", output["choices"][0]["text"])


In [ ]:
output = llm("¿Cuál es la capital de Italia?", max_tokens=32)
print(output["choices"][0]["text"])
